### 数据加载

In [ ]:
from source.load_excel_to_table import load_excel_to_table

part_a_conn = load_excel_to_table("/Users/sqjian/Documents/codespace/financial_tools/testdata/test.xlsx", "甲方", "part_a")
part_b_conn = load_excel_to_table("/Users/sqjian/Documents/codespace/financial_tools/testdata/test.xlsx", "乙方", "part_b")

### 提取分类条件

In [ ]:
import rich
from source.get_group_conditions import get_group_conditions


part_a_group_col_rst = get_group_conditions(part_a_conn, "part_a", "统一仓库", "统一名称", "统一日期")
part_b_group_col_rst = get_group_conditions(part_b_conn, "part_b", "统一仓库", "统一名称", "统一日期")

rich.print(part_a_group_col_rst)

### 分组提取匹配情况

In [ ]:
import rich
from source.reconcile_lists import reconcile_lists
from source.get_group_data import get_group_data

group1_exit = False
group2_exit = False

for group1 in part_a_group_col_rst["group1_col"]:
    if group1_exit:
        break
    group1_exit = True
    for group2 in part_a_group_col_rst["group2_col"]:
        if group2_exit:
            break
        group2_exit = True
        for group3 in part_a_group_col_rst["group3_col"]:
            list_a = get_group_data(part_a_conn, "part_a", "统一仓库", group1, "统一名称", group2, "统一日期", group3, "收货数量")
            list_b = get_group_data(part_b_conn, "part_b", "统一仓库", group1, "统一名称", group2, "统一日期", group3, "实发数量")
            rst = reconcile_lists(list_a, list_b)
            if all(len(value) == 0 for value in rst.values()):
                continue
            rich.print(f"{group1}=>{group2}=>{group3}", rst)

### 单点测试

In [ ]:
import rich
from source.reconcile_lists import reconcile_lists
from source.get_group_data import get_group_data
from collections import Counter

list_a = get_group_data(part_a_conn, "part_a", "统一仓库", "安徽合肥仓", "统一名称", "徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN", "统一日期", "2024-07-01 00:00:00", "收货数量")
list_b = get_group_data(part_b_conn, "part_b", "统一仓库", "安徽合肥仓", "统一名称", "徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN", "统一日期", "2024-07-01 00:00:00", "实发数量")
_rst = reconcile_lists(list_a, list_b)
rich.print(_rst)

In [ ]:
print(Counter(_rst["exact_matches"]))
print(Counter(_rst["exact_matches"]))
print(Counter(_rst["unmatched_a"]))
print(Counter(_rst["unmatched_b"]))

In [8]:
import rich


def reconcile_lists(list_a, list_b):
    """实现数值对账功能，包含精确匹配和双向组合匹配，以及多对多匹配

    Args:
        list_a: 数值列表A (e.g. [10.0, 20.0, 30.0])
        list_b: 数值列表B (e.g. [15.0, 25.0, 35.0])

    Returns:
        匹配结果字典
    """
    EPSILON = 1e-7

    # 使用Counter直接计数
    a_counter = Counter(list_a)
    b_counter = Counter(list_b)

    # 处理精确匹配
    exact_matches = {}
    common_nums = set(a_counter) & set(b_counter)
    for num in common_nums:
        match_count = min(a_counter[num], b_counter[num])
        if match_count > 0:
            exact_matches[num] = match_count
            a_counter[num] -= match_count
            b_counter[num] -= match_count

    # 只保留计数大于0的项
    a_counter = {k: v for k, v in a_counter.items() if v > 0}
    b_counter = {k: v for k, v in b_counter.items() if v > 0}

    # 生成剩余数值列表
    remaining_a = [num for num, count in a_counter.items() for _ in range(count)]
    remaining_b = [num for num, count in b_counter.items() for _ in range(count)]

    # 辅助函数：使用动态规划查找和为target的组合
    def find_combination_dp(candidates, target):
        if not candidates or abs(target) < EPSILON:
            return [] if abs(target) < EPSILON else None

        candidates.sort(reverse=True)
        memo = {}

        def dp(i, remaining):
            if abs(remaining) < EPSILON:
                return []
            if i >= len(candidates) or remaining < 0 - EPSILON:
                return None

            key = (i, round(remaining, 7))
            if key in memo:
                return memo[key]

            # 不选当前元素
            skip = dp(i + 1, remaining)

            # 选当前元素
            take = None
            if candidates[i] <= remaining + EPSILON:
                rest = dp(i + 1, remaining - candidates[i])
                if rest is not None:
                    take = [candidates[i]] + rest

            # 返回长度较小的那个非None结果
            result = None
            if take is not None and skip is not None:
                result = take if len(take) <= len(skip) else skip
            else:
                result = take if take is not None else skip

            memo[key] = result
            return result

        return dp(0, target)

    # 新增: 多对多匹配功能
    # 先尝试找到sum(a_subset) == sum(b_subset)的情况
    def find_equal_sum_subsets():
        many_to_many_matches = []

        # 获取所有可能的A列表组合的和值
        a_sums = {}
        for size in range(2, min(5, len(remaining_a) + 1)):  # 限制组合大小以控制复杂度
            for combo in generate_combinations(remaining_a, size):
                combo_sum = sum(combo)
                a_sums.setdefault(combo_sum, []).append(combo)

        # 获取所有可能的B列表组合的和值
        b_sums = {}
        for size in range(2, min(5, len(remaining_b) + 1)):
            for combo in generate_combinations(remaining_b, size):
                combo_sum = sum(combo)
                b_sums.setdefault(combo_sum, []).append(combo)

        # 找到相同的和值
        common_sums = set(a_sums.keys()) & set(b_sums.keys())

        # 处理所有匹配的情况
        for sum_val in sorted(common_sums, reverse=True):
            for a_combo in a_sums[sum_val]:
                for b_combo in b_sums[sum_val]:
                    # 检查这些元素是否还可用（未被匹配）
                    if all(val in remaining_a for val in a_combo) and all(val in remaining_b for val in b_combo):
                        many_to_many_matches.append({"type": "many_to_many", "a_values": a_combo, "b_values": b_combo, "sum": sum_val})

                        # 从剩余列表中移除这些已匹配的元素
                        for val in a_combo:
                            remaining_a.remove(val)
                        for val in b_combo:
                            remaining_b.remove(val)

                        # 一旦找到匹配就处理下一个和值
                        break

                # 如果已经找到匹配，处理下一个和值
                if any(match["sum"] == sum_val for match in many_to_many_matches):
                    break

        return many_to_many_matches

    # 辅助函数：生成列表的所有组合
    def generate_combinations(items, size):
        if size == 0:
            yield []
        elif items:
            # 包含第一个元素的组合
            first = items[0]
            for combo in generate_combinations(items[1:], size - 1):
                yield [first] + combo

            # 不包含第一个元素的组合
            for combo in generate_combinations(items[1:], size):
                yield combo

    # 进行常规的组合匹配
    # 复制列表用于组合匹配
    temp_a = sorted(remaining_a, reverse=True)
    temp_b = sorted(remaining_b, reverse=True)

    # 创建值到索引的映射
    a_indices = {v: i for i, v in enumerate(temp_a)}
    b_indices = {v: i for i, v in enumerate(temp_b)}

    # 组合匹配结果
    combo_a_to_b = []
    combo_b_to_a = []

    # 1. A 到 B 的组合匹配
    for b_val in sorted(set(temp_b), reverse=True):
        if b_val not in b_indices:
            continue

        b_idx = b_indices[b_val]
        current_a = [a for a in temp_a]

        combination = find_combination_dp(current_a, b_val)
        if combination:
            combo_a_to_b.append({"b": b_val, "a": combination})

            # 移除已匹配的值
            del temp_b[b_idx]
            b_indices = {v: i for i, v in enumerate(temp_b)}

            for a_val in combination:
                a_idx = temp_a.index(a_val)
                del temp_a[a_idx]

            # 更新索引映射
            a_indices = {v: i for i, v in enumerate(temp_a)}

    # 2. B 到 A 的组合匹配
    for a_val in sorted(set(temp_a), reverse=True):
        if a_val not in a_indices:
            continue

        a_idx = a_indices[a_val]
        current_b = [b for b in temp_b]

        combination = find_combination_dp(current_b, a_val)
        if combination:
            combo_b_to_a.append({"a": a_val, "b": combination})

            # 移除已匹配的值
            del temp_a[a_idx]
            a_indices = {v: i for i, v in enumerate(temp_a)}

            for b_val in combination:
                b_idx = temp_b.index(b_val)
                del temp_b[b_idx]

            # 更新索引映射
            b_indices = {v: i for i, v in enumerate(temp_b)}

    # 最后进行多对多匹配，优先级最低
    # 注意：我们需要使用temp_a和temp_b，因为这些是经过前面的匹配后剩余的元素
    remaining_a = temp_a
    remaining_b = temp_b
    many_to_many_matches = find_equal_sum_subsets()

    return {
        "exact_matches": exact_matches,
        "combo_matches_a_to_b": combo_a_to_b,
        "combo_matches_b_to_a": combo_b_to_a,
        "many_to_many_matches": many_to_many_matches,
        "unmatched_a": remaining_a,
        "unmatched_b": remaining_b,
    }


if __name__ == "__main__":
    # 测试案例1: 基础场景
    list_a = [20.0, 30.0, 50.0, 50.0]
    list_b = [10.0, 20.0, 30.0, 40.0]
    result = reconcile_lists(list_a, list_b)
    rich.print(result)

    # 测试案例2: 包含重复值
    list_a = [30.0, 15.0, 30.0, 15.0]
    list_b = [15.0, 15.0, 30.0, 9.99, 5.01]
    result = reconcile_lists(list_a, list_b)
    rich.print(result)

    # 测试案例3: 双向匹配
    list_a = [20.0, 30.0, 50.0, 50.0, 15, 2]
    list_b = [10.0, 20.0, 30.0, 40.0, 65]
    result = reconcile_lists(list_a, list_b)
    rich.print(result)

    # 测试案例4: 一边完全为空
    list_a = [10.0, 20.0, 30.0, 40.0, 65]
    list_b = []
    result = reconcile_lists(list_a, list_b)
    rich.print(result)

    # 测试案例5: 两遍双向匹配
    list_a = [45.0, 65, 1, 2, 3]
    list_b = [105, 1, 5, 5]
    result = reconcile_lists(list_a, list_b)
    rich.print(result)

{
    'exact_matches': {20.0: 1, 30.0: 1},
    'combo_matches_a_to_b': [],
    'combo_matches_b_to_a': [{'a': 50.0, 'b': [40.0, 10.0]}],
    'many_to_many_matches': [],
    'unmatched_a': [50.0],
    'unmatched_b': []
}

{
    'exact_matches': {30.0: 1, 15.0: 2},
    'combo_matches_a_to_b': [],
    'combo_matches_b_to_a': [],
    'many_to_many_matches': [],
    'unmatched_a': [30.0],
    'unmatched_b': [9.99, 5.01]
}

{
    'exact_matches': {20.0: 1, 30.0: 1},
    'combo_matches_a_to_b': [{'b': 65, 'a': [50.0, 15]}],
    'combo_matches_b_to_a': [{'a': 50.0, 'b': [40.0, 10.0]}],
    'many_to_many_matches': [],
    'unmatched_a': [2],
    'unmatched_b': []
}

{
    'exact_matches': {},
    'combo_matches_a_to_b': [],
    'combo_matches_b_to_a': [],
    'many_to_many_matches': [],
    'unmatched_a': [65, 40.0, 30.0, 20.0, 10.0],
    'unmatched_b': []
}

{
    'exact_matches': {1: 1},
    'combo_matches_a_to_b': [{'b': 5, 'a': [3, 2]}],
    'combo_matches_b_to_a': [],
    'many_to_many_matches': [{'type': 'many_to_many', 'a_values': [65, 45.0], 'b_values': [105, 5], 'sum': 110}],
    'unmatched_a': [],
    'unmatched_b': []
}